In [102]:
import os
import json
import pandas as pd
import traceback
import streamlit as st

In [103]:
from langchain.chat_models import ChatOpenAI

In [104]:
from dotenv import load_dotenv

load_dotenv()

True

In [105]:
KEY=st.secrets.get("OPENAI_API_KEY")

In [106]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [107]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [108]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [109]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [110]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [111]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [112]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [113]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [114]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [115]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [116]:
file_path=r"D:\mcqgenerator\data.txt"

In [117]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [118]:
print(TEXT)

### Machine Learning: An Overview

#### Introduction to Machine Learning
Machine Learning (ML) is a branch of artificial intelligence (AI) that focuses on the development of algorithms that allow computers to learn and make decisions based on data. Unlike traditional programming, where specific instructions are coded by humans, machine learning algorithms learn from data to make predictions or decisions without being explicitly programmed for the task.

#### Types of Machine Learning
Machine Learning is broadly categorized into three types:

1. **Supervised Learning:** In supervised learning, the algorithm is trained on labeled data. This means the input data is paired with the correct output. The goal is for the algorithm to learn a mapping from inputs to outputs and make predictions on unseen data. Common applications include image classification, spam detection, and predictive analytics.

2. **Unsupervised Learning:** Unsupervised learning involves training on data without labeled r

In [120]:
NUMBER=5 
SUBJECT="Machine learning"
TONE="simple"

In [121]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:### Machine Learning: An Overview

#### Introduction to Machine Learning
Machine Learning (ML) is a branch of artificial intelligence (AI) that focuses on the development of algorithms that allow computers to learn and make decisions based on data. Unlike traditional programming, where specific instructions are coded by humans, machine learning algorithms learn from data to make predictions or decisions without being explicitly programmed for the task.

#### Types of Machine Learning
Machine Learning is broadly categorized into three types:

1. **Supervised Learning:** In supervised learning, the algorithm is trained on labeled data. This means the input data is paired with the correct output. The goal is for the algorithm to learn a mapping from inputs to outputs and make predictions on unseen data. Common applications include image classification, spam detection, and predictive

In [122]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3117
Prompt Tokens:2376
Completion Tokens:741
Total Cost:0.005046


In [123]:
response

{'text': '### Machine Learning: An Overview\n\n#### Introduction to Machine Learning\nMachine Learning (ML) is a branch of artificial intelligence (AI) that focuses on the development of algorithms that allow computers to learn and make decisions based on data. Unlike traditional programming, where specific instructions are coded by humans, machine learning algorithms learn from data to make predictions or decisions without being explicitly programmed for the task.\n\n#### Types of Machine Learning\nMachine Learning is broadly categorized into three types:\n\n1. **Supervised Learning:** In supervised learning, the algorithm is trained on labeled data. This means the input data is paired with the correct output. The goal is for the algorithm to learn a mapping from inputs to outputs and make predictions on unseen data. Common applications include image classification, spam detection, and predictive analytics.\n\n2. **Unsupervised Learning:** Unsupervised learning involves training on da

In [124]:
quiz=response.get("quiz")

In [125]:
quiz=json.loads(quiz)

In [126]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [127]:
quiz_table_data

[{'MCQ': 'What is the goal of supervised learning in machine learning?',
  'Choices': 'a: To learn the underlying structure of data | b: To make predictions on unseen data based on labeled data | c: To interact with an environment and maximize cumulative rewards | d: To automate the model selection process',
  'Correct': 'b'},
 {'MCQ': 'Which type of learning involves training on data without labeled responses?',
  'Choices': 'a: Supervised Learning | b: Unsupervised Learning | c: Reinforcement Learning | d: Feature Engineering',
  'Correct': 'b'},
 {'MCQ': 'What is the purpose of feature engineering in machine learning?',
  'Choices': 'a: To automate the model selection process | b: To make predictions on unseen data | c: To select, modify, or create new variables from raw data | d: To maximize cumulative rewards',
  'Correct': 'c'},
 {'MCQ': 'Which machine learning algorithm is used for classification tasks by splitting data into branches based on feature values?',
  'Choices': 'a: L

In [128]:
quiz=pd.DataFrame(quiz_table_data)

In [129]:
quiz.to_csv("machinelearning.csv",index=False)